In [1]:
%matplotlib inline

from GeoToolkit.Mag import Mag, ProblemSetter, MathUtils, Simulator, DataIO
import matplotlib.pyplot as plt
from GeoToolkit.graphics import graphics
import numpy as np
import scipy.sparse as sp
from scipy.spatial import Delaunay
from skimage import exposure
from geoh5io.workspace import Workspace
from geoh5io.objects import Surface

In [2]:
download = DataIO.fetchData(path=r"C:\Users\DominiqueFournier\Documents\GIT\GeoToolkit\Notebooks\Output", localCloud='Local')
display(download)

interactive(children=(RadioButtons(description='File Type:', options=('Local', 'Cloud'), value='Local'), Text(…

In [5]:
if download.result is not None:
    if (download.result[1] == "CSV")  :
        grid = Simulator.dataGriddingWidget(download.result[0], marker=False, dataColumn=3, Resolution=100, Method='linear')
        display(grid)
    else:
        grid = Simulator.dataGridGeoref(download.result[0])
        display(grid)

interactive(children=(Dropdown(description='ColorMap', index=3, options=('viridis', 'plasma', 'magma', 'Spectr…

In [7]:
if download.result is not None:
    selection = Simulator.setDataExtentWidget(grid.result)
    display(selection)

interactive(children=(FloatSlider(value=-5.389099242057903e-15, continuous_update=False, description='East', m…

In [8]:

sub = 5

grid_shape = selection.result.values.shape
values = (selection.result.values.copy())
indNan = np.isnan(values)
values[indNan] = 0
values = values[::sub, ::sub].ravel()


theta = selection.result.gridCC[:, 0].reshape(grid_shape, order='F')[::sub, ::sub].ravel()
phi = selection.result.gridCC[:, 1].reshape(grid_shape, order='F')[::sub, ::sub].ravel()


R = 5380
R = (R+(values)/10)
x = R * np.cos(np.deg2rad(phi)) * np.cos(np.deg2rad(theta))
y = R * np.cos(np.deg2rad(phi)) * np.sin(np.deg2rad(theta))
z = R * np.sin(np.deg2rad(phi))


# Triangulate the surface
surf = Delaunay(np.c_[theta, phi])

In [9]:
# Interpolate values on cell
values = np.mean(np.c_[
                     values[surf.simplices[:, 0]],
                     values[surf.simplices[:, 1]],
                     values[surf.simplices[:, 2]]
                 ], axis=1
)



In [11]:
# Create a geoh5 surface and export
workspace = Workspace("EMAG_V2_new2.geoh5")
surface, data = Surface.create(
    workspace, vertices=np.c_[x, y, z], 
    name="Tester",
    cells=surf.simplices,
    data={"TMI": ["CELL", values]})
workspace.save_entity(surface)
workspace.finalize()


In [12]:
workspace.tree??


In [25]:
obj = workspace.get_entity("CentralCanada")[0]
obj.name = "SouthAmerica"

workspace.save_entity(obj)
workspace.finalize()

# workspace.tree


In [31]:
aa = [1]



if aa:
    
    print('Is true')


Is true


# Repeat for EWGM2012 (GRAV)